# Massive Machine Learning Pipelines - Part 3
### Cross Validation

It's pretty disappointing to see all that hard work separating columns into groups doing absolutely nothing for our score. Since Kaggle only allows 10 submissions per day, it would have been nice to know that our model wouldn't show an improvement before sending our submission.

We can get a fairly good idea of how our model is going to perform by doing cross validation. Below, we perform K-Fold cross validation with 5 folds. By default, the r-squared value will be returned as the score. Instead, we can return the negative mean of the squared log error, which is nearly the same error calculation that Kaggle uses.

In most all other machine learning contexts, the root mean squared error is reported as a positive number. scikit-learn reports it as a negative number. This is strange, but is done because it considers a greater score to be better and ranks models based on their score when grid searching.

In [ ]:
# warning - this might not work for other random states
from sklearn.model_selection import cross_val_score, KFold
kf = KFold(n_splits=5, shuffle=True, random_state=123)
scores = cross_val_score(final_pipe2, housing, y, 
                         cv=kf, scoring='neg_mean_squared_log_error')
scores

To report the same metric as Kaggle, we will negate the scores, take the square root, and then average all of the scores. Because we are shuffling the data during cross validation, you'll get a different score each time, but it should be around .18, which indicates that there would be no improvement over the first model.

In [ ]:
import numpy as np
np.mean(np.sqrt(-scores))

### Transforming the target variable
Instead of using the actual sale price as the target variable, we can instead use the log of it. This is common practice when modeling data that with a large range of values. Kaggle is already scoring based on the log of the sale price so it makes sense to use it directly. 

scikit-learn provides the `TransformedTargetRegressor` meta-estimator to automate the transforming of any continuous target variable. We pass it our pipeline, the transformation function applied to the target and a function to invert the transformation. The model will be trained against the transformed target variable (the log of the sale price in our example). When predicting, the inverse function will be used to transform the target back to the original units.

Let's use this new estimator to train our model and make predictions.

In [ ]:
from sklearn.compose import TransformedTargetRegressor
final_pipe2_ttr = TransformedTargetRegressor(final_pipe2, func=np.log, inverse_func=np.exp)
final_pipe2_ttr.fit(housing, y)
final_pipe2_ttr.predict(housing_test)

Now that we verified the model can make predictions, let's use cross validation to get a estimate a score.

In [ ]:
scores = cross_val_score(final_pipe2_ttr, housing, y, 
                         cv=kf, scoring='neg_mean_squared_log_error')
np.mean(np.sqrt(-scores))

The score should have dropped to around .16 which gives us confidence that we will have some improvement over the original. Let's make our submission to Kaggle.

In [ ]:
file = 'data/submissions/20190710/sub03.csv'
message = 'Same as 2, but trainined on log of y.'
# submit_kaggle(final_pipe2_ttr, housing_test, file, message)

Our score from kaggle is .14 which is better than what was predicted by cross validation and far better than our original

In [ ]:
all_submissions = kaggle.api.competitions_submissions_list(competition)
all_submissions[0]

### Overfitting
Our model with all of the columns produced a similar score as the first trained on just a fraction of the columns when modeled against the actual sales price. This is likely due to the extreme flexibility that we have given the model. We used all 79 variables and one-hot encoded most of them, which created an enormous number of features. Let's see exactly how many features were created by running our data just through the `ColumnTransformer`.

In [ ]:
ct.transform(housing).shape

We built our model on input data with over 600 features or nearly half the total number of observations. If we are to continue to use this high number of features, we'll need to choose a different type of model or apply constraints to this one.

### Regularization
Ridge regression constrains the parameters of the model by applying a penalty term (`alpha`) preventing the model from learning too much from the training data. We use the `GridSearchCV` meta-estimator to find the best possible `alpha` from 100 given between .01 and 1000. By default, `alpha` is set to 1.

In [ ]:
from sklearn.model_selection import GridSearchCV
param_grid = {'regressor__ridge__alpha': np.logspace(-2, 3, 100)}
gs = GridSearchCV(final_pipe2_ttr, param_grid, cv=kf, scoring='neg_mean_squared_log_error')
gs.fit(housing, y);

Let's get the best value for alpha.

In [ ]:
gs.best_params_

And the score associated with that alpha which is still right around .15.

In [ ]:
np.sqrt(-gs.best_score_)

We can assign the best model trained to its own variable and then get predictions.

In [ ]:
final_pipe2_ttr_best = gs.best_estimator_
y_pred = final_pipe2_ttr_best.predict(housing_test)
y_pred

Even with the best value for alpha, our ridge regression produced just about the same score. We won't bother submitting it to Kaggle. Let's try reducing the total number of features instead.